# Random Forest Classifier for 1064nm Data

In [1]:
import os
import glob
import re
from collections import defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import sparse
from scipy.sparse.linalg import spsolve

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_recall_fscore_support

## ZEA

-Random forest <br>
-Full spectrum and 40 partial spectra <br>
-4 measurements of the same sample were averaged <br>
-Baseline subtracted <br>
-Normalization: none, max, integral

In [ ]:
# ----------------------------
# Filename and Parameters
# ----------------------------
file_pattern = "P*_A*_D*_Z*_F*_1064nm_*mW_*ms_*av_no*.csv"  
limit_Z = 40  # Threshold for Zea
random_state = 42
n_estimators = 200

output_dir = "Results_Random-Forest_sampels-averaged_Zea_1064nm"  #Defines folder for results
os.makedirs(output_dir, exist_ok=True)

# ----------------------------
# Baseline ALS 
# ----------------------------
def baseline_als(y, lam=1e5, p=0.001, niter=10):
    L = len(y)
    D = sparse.diags([1, -2, 1], [0, 1, 2], shape=(L-2, L))
    w = np.ones(L)
    for i in range(niter):
        W = sparse.spdiags(w, 0, L, L)
        Z = W + lam * (D.T @ D)
        z = spsolve(Z, w * y)
        w = p * (y > z) + (1.0 - p) * (y < z)
    return z

# ----------------------------
# Extraction P- and Z-Values
# ----------------------------
def extract_P_value_from_filename(fname):
    base = os.path.basename(fname)
    m = re.search(r'(P\d+)', base)
    return m.group(1) if m else None

def extract_Z_from_filename(fname):
    base = os.path.basename(fname)
    parts = base.replace(".csv","").split("_")
    for token in parts:
        if token.startswith("Z"):
            try:
                return float(token[1:])
            except:
                pass
    return None

# ----------------------------
# Collect files and group by P
# ----------------------------
files = sorted(glob.glob(file_pattern))
print(f"{len(files)} files found by pattern '{file_pattern}'.")

groups = defaultdict(list)
for f in files:
    pid = extract_P_value_from_filename(f)
    if pid is None:
        print("Warning: File without P-token:", f)
        continue
    groups[pid].append(f)

print(f"{len(groups)} P groups found.")

# ----------------------------
# Load spectra, baseline-remove, average per group
# ----------------------------
group_spectra = {}
for pid, filelist in groups.items():
    spectra_list = []
    wav = None
    label = None
    for f in filelist:
        df = pd.read_csv(f, skiprows=56, header=None)
        try:
            df = df[[2,3]].apply(pd.to_numeric, errors='coerce').dropna()
            df.columns = ["Wavenumber", "Processed"]
        except Exception as e:
            raise RuntimeError(f"Error reading {f}: {e}")
        wavenumbers = df["Wavenumber"].values
        proc = df["Processed"].values.astype(float)

        if wav is None:
            wav = wavenumbers
        else:
            if not np.allclose(wav, wavenumbers):
                proc = np.interp(wav, wavenumbers, proc)

        baseline = baseline_als(proc, lam=1e3, p=0.001, niter=10)
        proc_corr = proc - baseline
        spectra_list.append(proc_corr)

        if label is None:
            zval = extract_Z_from_filename(f)
            if zval is None:
                raise RuntimeError(f"No Z specification in file name {f}.")
            label = int(zval > limit_Z)

    spectra_arr = np.vstack(spectra_list)
    mean_spec = spectra_arr.mean(axis=0)
    group_spectra[pid] = {'wavenumber': wav, 'spectrum': mean_spec, 'label': label, 'n_original_files': len(filelist)}

print(f"{len(group_spectra)} averaged group spectra are generated.")

# ----------------------------
# Prepare regions: full + 20 windows
# ----------------------------
any_pid = next(iter(group_spectra))
L = len(group_spectra[any_pid]['spectrum'])
windows = [('full', 0, L)]
n_win = 20
inds = np.round(np.linspace(0, L, n_win+1)).astype(int)
for i in range(n_win):
    windows.append((f'win_{i+1}', inds[i], inds[i+1]))

print(f"Total regions: {len(windows)} (1 full + {n_win} windows). Each vector length: {L}")

# ----------------------------
# Preprocessing modes
# ----------------------------
def preprocess_vector(v, mode):
    if mode == 'none':
        return v.copy()
    elif mode == 'max':
        m = np.max(np.abs(v))
        if m == 0:
            return v.copy()
        return v / m
    elif mode == 'integral':
        s = np.sum(np.abs(v))
        if s == 0:
            return v.copy()
        return v / s
    else:
        raise ValueError("Unknown mode")

preproc_modes = ['none', 'max', 'integral']

# ----------------------------
# Build dataset arrays
# ----------------------------
pids_sorted = sorted(group_spectra.keys())
data_by_pid = {}
for pid in pids_sorted:
    data_by_pid[pid] = {
        'wavenumber': group_spectra[pid]['wavenumber'],
        'spectrum': group_spectra[pid]['spectrum'],
        'label': group_spectra[pid]['label'],
        'n_files': group_spectra[pid]['n_original_files']
    }

# ----------------------------
# Run experiments whit LOOCV
# ----------------------------
results = []
exp_id = 0

for mode in preproc_modes:
    for region_name, start_idx, end_idx in windows:
        exp_id += 1
        X_list, y_list, infos = [], [], []
        for pid in pids_sorted:
            spec = data_by_pid[pid]['spectrum'][start_idx:end_idx]
            spec_p = preprocess_vector(spec, mode)
            if spec_p.size == 0:
                continue
            X_list.append(spec_p)
            y_list.append(data_by_pid[pid]['label'])
            infos.append((pid, data_by_pid[pid]['n_files']))
        X_arr = np.vstack(X_list)
        y_arr = np.array(y_list)
        n_samples, n_features = X_arr.shape[0], X_arr.shape[1]

        if n_samples < 2:
            print(f"Experiment {exp_id} ({mode}, {region_name}) skipped: less samples ({n_samples}).")
            continue

        clf = RandomForestClassifier(n_estimators=n_estimators, random_state=random_state)

        # LOOCV
        loo = LeaveOneOut()
        y_true_all, y_pred_all = [], []
        for train_idx, test_idx in loo.split(X_arr):
            X_train, X_test = X_arr[train_idx], X_arr[test_idx]
            y_train, y_test = y_arr[train_idx], y_arr[test_idx]
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            y_true_all.extend(y_test.tolist())
            y_pred_all.extend(y_pred.tolist())

        # metrics
        acc = accuracy_score(y_true_all, y_pred_all)
        prec, rec, f1, _ = precision_recall_fscore_support(y_true_all, y_pred_all, average='macro', zero_division=0)
        cls_report = classification_report(y_true_all, y_pred_all, zero_division=0)
        cm = confusion_matrix(y_true_all, y_pred_all)

        # Raman-Bereich
        wavenumber = data_by_pid[pids_sorted[0]]['wavenumber']
        raman_start = wavenumber[start_idx]
        raman_end = wavenumber[end_idx-1]
        raman_range_str = f"{raman_start:.1f}–{raman_end:.1f} cm⁻¹"

        # save report & CM figure
        exp_name = f"exp_{exp_id:03d}_{mode}_{region_name}"
        rep_file = os.path.join(output_dir, f"report_{exp_name}.txt")
        with open(rep_file, "w", encoding="utf-8") as fh:
            fh.write(f"Experiment: {exp_name}\n")
            fh.write(f"Normalization: {mode}\n")
            fh.write(f"Region: {region_name} (indices {start_idx}:{end_idx})\n")
            fh.write(f"RamanRegion: {raman_range_str}\n")
            fh.write(f"n_samples: {n_samples}, n_features: {n_features}\n\n")
            fh.write(f"Accuracy: {acc:.4f}\n")
            fh.write(f"Precision (macro): {prec:.4f}\n")
            fh.write(f"Recall (macro): {rec:.4f}\n")
            fh.write(f"F1 (macro): {f1:.4f}\n\n")
            fh.write("Classification report:\n")
            fh.write(cls_report)
            fh.write("\nConfusion matrix (raw):\n")
            fh.write(np.array2string(cm))

        plt.figure(figsize=(6,5))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                    xticklabels=[f"Z <= {limit_Z}", f"Z > {limit_Z}"],
                    yticklabels=[f"Z <= {limit_Z}", f"Z > {limit_Z}"])
        plt.xlabel("Predicted label")
        plt.ylabel("True label")
        plt.title(
            f"RandomForest LOOCV\n"
            f"Excitation wavelength: 1064nm\n"
            f"Normalization: {mode}\n"
            f"Raman region: {raman_range_str}\n"
            f"Sample averaging: yes\n"
            f"Accuracy: {acc:.4f}"
        )
        plt.tight_layout()
        cm_file = os.path.join(output_dir, f"cm_{exp_name}.png")
        plt.savefig(cm_file)
        plt.close()

        # collect summary
        results.append({
            'exp_id': exp_id,
            'exp_name': exp_name,
            'Normalization': mode,
            'region': region_name,
            'region_start': int(start_idx),
            'region_end': int(end_idx),
            'raman_start': float(raman_start),
            'raman_end': float(raman_end),
            'n_samples': int(n_samples),
            'n_features': int(n_features),
            'accuracy': float(acc),
            'precision_macro': float(prec),
            'recall_macro': float(rec),
            'f1_macro': float(f1),
            'report_file': rep_file,
            'cm_file': cm_file
        })

        print(f"[{exp_id:03d}] mode={mode} region={region_name} samples={n_samples} features={n_features} acc={acc:.3f}")

# ----------------------------
# Save summary CSV
# ----------------------------
df_results = pd.DataFrame(results)
summary_csv = os.path.join(output_dir, "results_summary_loocv.csv")
df_results.to_csv(summary_csv, index=False)
print("Summary saved to:", summary_csv)
print("Individual reports and confusion matrices in the directory:", output_dir)


80 files found by pattern 'P*_A*_D*_Z*_F*_1064nm_*mW_*ms_*av_no*.csv'.
20 P groups found.


C:\Users\miles\AppData\Local\Temp\ipykernel_24876\707442820.py:22: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  z = spsolve(Z, w * y)
C:\Users\miles\AppData\Local\Temp\ipykernel_24876\707442820.py:22: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  z = spsolve(Z, w * y)
C:\Users\miles\AppData\Local\Temp\ipykernel_24876\707442820.py:22: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  z = spsolve(Z, w * y)
C:\Users\miles\AppData\Local\Temp\ipykernel_24876\707442820.py:22: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  z = spsolve(Z, w * y)
C:\Users\miles\AppData\Local\Temp\ipykernel_24876\707442820.py:22: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  z = spsolve(Z, w * y)
C:\Users\miles\AppData\Local\Temp\ipykernel_24876\707442820.py:22: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  z = spsolve(Z, w * y)
C:\Users\miles\AppData

20 averaged group spectra are generated.
Total regions: 21 (1 full + 20 windows). Each vector length: 512
[001] mode=none region=full samples=20 features=512 acc=0.700
[002] mode=none region=win_1 samples=20 features=26 acc=0.750
[003] mode=none region=win_2 samples=20 features=25 acc=0.850
[004] mode=none region=win_3 samples=20 features=26 acc=0.750
[005] mode=none region=win_4 samples=20 features=25 acc=0.800
[006] mode=none region=win_5 samples=20 features=26 acc=0.750
[007] mode=none region=win_6 samples=20 features=26 acc=0.750
[008] mode=none region=win_7 samples=20 features=25 acc=0.700
[009] mode=none region=win_8 samples=20 features=26 acc=0.700
[010] mode=none region=win_9 samples=20 features=25 acc=0.700
[011] mode=none region=win_10 samples=20 features=26 acc=0.600
[012] mode=none region=win_11 samples=20 features=26 acc=0.450
[013] mode=none region=win_12 samples=20 features=25 acc=0.650
[014] mode=none region=win_13 samples=20 features=26 acc=0.600
[015] mode=none region

## AFLA

-Random forest <br>
-Full spectrum and 40 partial spectra <br>
-4 measurements of the same sample were averaged <br>
-Baseline subtracted <br>
-Normalization: none, max, integral

In [3]:
# ----------------------------
# Filename and Parameters
# ----------------------------
file_pattern = "P*_A*_D*_Z*_F*_1064nm_*mW_*ms_*av_no*.csv"  
limit_A = 3  # Threshold for AFLA
random_state = 42
n_estimators = 200

output_dir = "Results_Random-Forest_sampels-averaged_Afla_1064nm"  #Defines folder for results
os.makedirs(output_dir, exist_ok=True)

# ----------------------------
# Baseline ALS 
# ----------------------------
def baseline_als(y, lam=1e5, p=0.001, niter=10):
    L = len(y)
    D = sparse.diags([1, -2, 1], [0, 1, 2], shape=(L-2, L))
    w = np.ones(L)
    for i in range(niter):
        W = sparse.spdiags(w, 0, L, L)
        Z = W + lam * (D.T @ D)
        z = spsolve(Z, w * y)
        w = p * (y > z) + (1.0 - p) * (y < z)
    return z

# ----------------------------
# Extraction P- and A-Values
# ----------------------------
def extract_P_value_from_filename(fname):
    base = os.path.basename(fname)
    m = re.search(r'(P\d+)', base)
    return m.group(1) if m else None

def extract_A_from_filename(fname):
    base = os.path.basename(fname)
    parts = base.replace(".csv","").split("_")
    for token in parts:
        if token.startswith("A"):
            try:
                return float(token[1:])
            except:
                pass
    return None

# ----------------------------
# Collect files and group by P
# ----------------------------
files = sorted(glob.glob(file_pattern))
print(f"{len(files)} files found by pattern '{file_pattern}'.")

groups = defaultdict(list)
for f in files:
    pid = extract_P_value_from_filename(f)
    if pid is None:
        print("Warning: File without P-token:", f)
        continue
    groups[pid].append(f)

print(f"{len(groups)} P groups found.")

# ----------------------------
# Load spectra, baseline-remove, average per group
# ----------------------------
group_spectra = {}
for pid, filelist in groups.items():
    spectra_list = []
    wav = None
    label = None
    for f in filelist:
        df = pd.read_csv(f, skiprows=56, header=None)
        try:
            df = df[[2,3]].apply(pd.to_numeric, errors='coerce').dropna()
            df.columns = ["Wavenumber", "Processed"]
        except Exception as e:
            raise RuntimeError(f"Error reading {f}: {e}")
        wavenumbers = df["Wavenumber"].values
        proc = df["Processed"].values.astype(float)

        if wav is None:
            wav = wavenumbers
        else:
            if not np.allclose(wav, wavenumbers):
                proc = np.interp(wav, wavenumbers, proc)

        baseline = baseline_als(proc, lam=1e3, p=0.001, niter=10)
        proc_corr = proc - baseline
        spectra_list.append(proc_corr)

        if label is None:
            aval = extract_A_from_filename(f)
            if aval is None:
                raise RuntimeError(f"No A specification in file name {f}.")
            label = int(aval > limit_A)

    spectra_arr = np.vstack(spectra_list)
    mean_spec = spectra_arr.mean(axis=0)
    group_spectra[pid] = {'wavenumber': wav, 'spectrum': mean_spec, 'label': label, 'n_original_files': len(filelist)}

print(f"{len(group_spectra)} averaged group spectra are generated.")

# ----------------------------
# Prepare regions: full + 40 windows
# ----------------------------
any_pid = next(iter(group_spectra))
L = len(group_spectra[any_pid]['spectrum'])
windows = [('full', 0, L)]
n_win = 40
inds = np.round(np.linspace(0, L, n_win+1)).astype(int)
for i in range(n_win):
    windows.append((f'win_{i+1}', inds[i], inds[i+1]))

print(f"Total regions: {len(windows)} (1 full + {n_win} windows). Each vector length: {L}")

# ----------------------------
# Preprocessing modes
# ----------------------------
def preprocess_vector(v, mode):
    if mode == 'none':
        return v.copy()
    elif mode == 'max':
        m = np.max(np.abs(v))
        if m == 0:
            return v.copy()
        return v / m
    elif mode == 'integral':
        s = np.sum(np.abs(v))
        if s == 0:
            return v.copy()
        return v / s
    else:
        raise ValueError("Unknown mode")

preproc_modes = ['none', 'max', 'integral']

# ----------------------------
# Build dataset arrays
# ----------------------------
pids_sorted = sorted(group_spectra.keys())
data_by_pid = {}
for pid in pids_sorted:
    data_by_pid[pid] = {
        'wavenumber': group_spectra[pid]['wavenumber'],
        'spectrum': group_spectra[pid]['spectrum'],
        'label': group_spectra[pid]['label'],
        'n_files': group_spectra[pid]['n_original_files']
    }

# ----------------------------
# Run experiments with LOOCV
# ----------------------------
results = []
exp_id = 0

for mode in preproc_modes:
    for region_name, start_idx, end_idx in windows:
        exp_id += 1
        X_list, y_list, infos = [], [], []
        for pid in pids_sorted:
            spec = data_by_pid[pid]['spectrum'][start_idx:end_idx]
            spec_p = preprocess_vector(spec, mode)
            if spec_p.size == 0:
                continue
            X_list.append(spec_p)
            y_list.append(data_by_pid[pid]['label'])
            infos.append((pid, data_by_pid[pid]['n_files']))
        X_arr = np.vstack(X_list)
        y_arr = np.array(y_list)
        n_samples, n_features = X_arr.shape[0], X_arr.shape[1]

        if n_samples < 2:
            print(f"Experiment {exp_id} ({mode}, {region_name}) skipped: less samples ({n_samples}).")
            continue

        clf = RandomForestClassifier(n_estimators=n_estimators, random_state=random_state)

        # LOOCV
        loo = LeaveOneOut()
        y_true_all, y_pred_all = [], []
        for train_idx, test_idx in loo.split(X_arr):
            X_train, X_test = X_arr[train_idx], X_arr[test_idx]
            y_train, y_test = y_arr[train_idx], y_arr[test_idx]
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            y_true_all.extend(y_test.tolist())
            y_pred_all.extend(y_pred.tolist())

        # metrics
        acc = accuracy_score(y_true_all, y_pred_all)
        prec, rec, f1, _ = precision_recall_fscore_support(y_true_all, y_pred_all, average='macro', zero_division=0)
        cls_report = classification_report(y_true_all, y_pred_all, zero_division=0)
        cm = confusion_matrix(y_true_all, y_pred_all)

        # Raman-Bereich
        wavenumber = data_by_pid[pids_sorted[0]]['wavenumber']
        raman_start = wavenumber[start_idx]
        raman_end = wavenumber[end_idx-1]
        raman_range_str = f"{raman_start:.1f}–{raman_end:.1f} cm⁻¹"

        # save report & CM figure
        exp_name = f"exp_{exp_id:03d}_{mode}_{region_name}"
        rep_file = os.path.join(output_dir, f"report_{exp_name}.txt")
        with open(rep_file, "w", encoding="utf-8") as fh:
            fh.write(f"Experiment: {exp_name}\n")
            fh.write(f"Normalization: {mode}\n")
            fh.write(f"Region: {region_name} (indices {start_idx}:{end_idx})\n")
            fh.write(f"RamanRegion: {raman_range_str}\n")
            fh.write(f"n_samples: {n_samples}, n_features: {n_features}\n\n")
            fh.write(f"Accuracy: {acc:.4f}\n")
            fh.write(f"Precision (macro): {prec:.4f}\n")
            fh.write(f"Recall (macro): {rec:.4f}\n")
            fh.write(f"F1 (macro): {f1:.4f}\n\n")
            fh.write("Classification report:\n")
            fh.write(cls_report)
            fh.write("\nConfusion matrix (raw):\n")
            fh.write(np.array2string(cm))

        plt.figure(figsize=(6,5))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                    xticklabels=[f"A <= {limit_A}", f"A > {limit_A}"],
                    yticklabels=[f"A <= {limit_A}", f"A > {limit_A}"])
        plt.xlabel("Predicted label")
        plt.ylabel("True label")
        plt.title(
            f"RandomForest LOOCV\n"
            f"Excitation wavelength: 1064nm\n"
            f"Normalization: {mode}\n"
            f"Raman region: {raman_range_str}\n"
            f"Sample averaging: yes\n"
            f"Accuracy: {acc:.4f}"
        )
        plt.tight_layout()
        cm_file = os.path.join(output_dir, f"cm_{exp_name}.png")
        plt.savefig(cm_file)
        plt.close()

        # collect summary
        results.append({
            'exp_id': exp_id,
            'exp_name': exp_name,
            'Normalization': mode,
            'region': region_name,
            'region_start': int(start_idx),
            'region_end': int(end_idx),
            'raman_start': float(raman_start),
            'raman_end': float(raman_end),
            'n_samples': int(n_samples),
            'n_features': int(n_features),
            'accuracy': float(acc),
            'precision_macro': float(prec),
            'recall_macro': float(rec),
            'f1_macro': float(f1),
            'report_file': rep_file,
            'cm_file': cm_file
        })

        print(f"[{exp_id:03d}] mode={mode} region={region_name} samples={n_samples} features={n_features} acc={acc:.3f}")

# ----------------------------
# Save summary CSV
# ----------------------------
df_results = pd.DataFrame(results)
summary_csv = os.path.join(output_dir, "results_summary_loocv.csv")
df_results.to_csv(summary_csv, index=False)
print("Summary saved to:", summary_csv)
print("Individual reports and confusion matrices in the directory:", output_dir)

80 files found by pattern 'P*_A*_D*_Z*_F*_1064nm_*mW_*ms_*av_no*.csv'.
20 P groups found.


C:\Users\miles\AppData\Local\Temp\ipykernel_20380\943639454.py:22: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  z = spsolve(Z, w * y)
C:\Users\miles\AppData\Local\Temp\ipykernel_20380\943639454.py:22: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  z = spsolve(Z, w * y)
C:\Users\miles\AppData\Local\Temp\ipykernel_20380\943639454.py:22: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  z = spsolve(Z, w * y)
C:\Users\miles\AppData\Local\Temp\ipykernel_20380\943639454.py:22: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  z = spsolve(Z, w * y)
C:\Users\miles\AppData\Local\Temp\ipykernel_20380\943639454.py:22: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  z = spsolve(Z, w * y)
C:\Users\miles\AppData\Local\Temp\ipykernel_20380\943639454.py:22: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  z = spsolve(Z, w * y)
C:\Users\miles\AppData

20 averaged group spectra are generated.
Total regions: 41 (1 full + 40 windows). Each vector length: 512
[001] mode=none region=full samples=20 features=512 acc=0.800
[002] mode=none region=win_1 samples=20 features=13 acc=0.850
[003] mode=none region=win_2 samples=20 features=13 acc=0.700
[004] mode=none region=win_3 samples=20 features=12 acc=0.800
[005] mode=none region=win_4 samples=20 features=13 acc=0.700
[006] mode=none region=win_5 samples=20 features=13 acc=0.700
[007] mode=none region=win_6 samples=20 features=13 acc=0.600
[008] mode=none region=win_7 samples=20 features=13 acc=0.700
[009] mode=none region=win_8 samples=20 features=12 acc=0.650
[010] mode=none region=win_9 samples=20 features=13 acc=0.700
[011] mode=none region=win_10 samples=20 features=13 acc=0.700
[012] mode=none region=win_11 samples=20 features=13 acc=0.600
[013] mode=none region=win_12 samples=20 features=13 acc=0.650
[014] mode=none region=win_13 samples=20 features=12 acc=0.750
[015] mode=none region

## DON

-Random forest <br>
-Full spectrum and 40 partial spectra <br>
-4 measurements of the same sample were averaged <br>
-Baseline subtracted <br>
-Normalization: none, max, integral

In [4]:
# ----------------------------
# Filename and Parameters
# ----------------------------
file_pattern = "P*_A*_D*_Z*_F*_1064nm_*mW_*ms_*av_no*.csv"  
limit_D = 1000  # Threshold for DON
random_state = 42
n_estimators = 200

output_dir = "Results_Random-Forest_sampels-averaged_DON_1064nm"  #Defines folder for results
os.makedirs(output_dir, exist_ok=True)

# ----------------------------
# Baseline ALS 
# ----------------------------
def baseline_als(y, lam=1e5, p=0.001, niter=10):
    L = len(y)
    D = sparse.diags([1, -2, 1], [0, 1, 2], shape=(L-2, L))
    w = np.ones(L)
    for i in range(niter):
        W = sparse.spdiags(w, 0, L, L)
        Z = W + lam * (D.T @ D)
        z = spsolve(Z, w * y)
        w = p * (y > z) + (1.0 - p) * (y < z)
    return z

# ----------------------------
# Extraction P- and D-Values
# ----------------------------
def extract_P_value_from_filename(fname):
    base = os.path.basename(fname)
    m = re.search(r'(P\d+)', base)
    return m.group(1) if m else None

def extract_D_from_filename(fname):
    base = os.path.basename(fname)
    parts = base.replace(".csv","").split("_")
    for token in parts:
        if token.startswith("D"):
            try:
                return float(token[1:])
            except:
                pass
    return None

# ----------------------------
# Collect files and group by P
# ----------------------------
files = sorted(glob.glob(file_pattern))
print(f"{len(files)} files found by pattern '{file_pattern}'.")

groups = defaultdict(list)
for f in files:
    pid = extract_P_value_from_filename(f)
    if pid is None:
        print("Warning: File without P-token:", f)
        continue
    groups[pid].append(f)

print(f"{len(groups)} P groups found.")

# ----------------------------
# Load spectra, baseline-remove, average per group
# ----------------------------
group_spectra = {}
for pid, filelist in groups.items():
    spectra_list = []
    wav = None
    label = None
    for f in filelist:
        df = pd.read_csv(f, skiprows=56, header=None)
        try:
            df = df[[2,3]].apply(pd.to_numeric, errors='coerce').dropna()
            df.columns = ["Wavenumber", "Processed"]
        except Exception as e:
            raise RuntimeError(f"Error reading {f}: {e}")
        wavenumbers = df["Wavenumber"].values
        proc = df["Processed"].values.astype(float)

        if wav is None:
            wav = wavenumbers
        else:
            if not np.allclose(wav, wavenumbers):
                proc = np.interp(wav, wavenumbers, proc)

        baseline = baseline_als(proc, lam=1e3, p=0.001, niter=10)
        proc_corr = proc - baseline
        spectra_list.append(proc_corr)

        if label is None:
            dval = extract_D_from_filename(f)
            if dval is None:
                raise RuntimeError(f"No D specification in file name {f}.")
            label = int(dval > limit_D)

    spectra_arr = np.vstack(spectra_list)
    mean_spec = spectra_arr.mean(axis=0)
    group_spectra[pid] = {'wavenumber': wav, 'spectrum': mean_spec, 'label': label, 'n_original_files': len(filelist)}

print(f"{len(group_spectra)} averaged group spectra are generated.")

# ----------------------------
# Prepare regions: full + 40 windows
# ----------------------------
any_pid = next(iter(group_spectra))
L = len(group_spectra[any_pid]['spectrum'])
windows = [('full', 0, L)]
n_win = 40
inds = np.round(np.linspace(0, L, n_win+1)).astype(int)
for i in range(n_win):
    windows.append((f'win_{i+1}', inds[i], inds[i+1]))

print(f"Total regions: {len(windows)} (1 full + {n_win} windows). Each vector length: {L}")

# ----------------------------
# Preprocessing modes
# ----------------------------
def preprocess_vector(v, mode):
    if mode == 'none':
        return v.copy()
    elif mode == 'max':
        m = np.max(np.abs(v))
        if m == 0:
            return v.copy()
        return v / m
    elif mode == 'integral':
        s = np.sum(np.abs(v))
        if s == 0:
            return v.copy()
        return v / s
    else:
        raise ValueError("Unknown mode")

preproc_modes = ['none', 'max', 'integral']

# ----------------------------
# Build dataset arrays
# ----------------------------
pids_sorted = sorted(group_spectra.keys())
data_by_pid = {}
for pid in pids_sorted:
    data_by_pid[pid] = {
        'wavenumber': group_spectra[pid]['wavenumber'],
        'spectrum': group_spectra[pid]['spectrum'],
        'label': group_spectra[pid]['label'],
        'n_files': group_spectra[pid]['n_original_files']
    }

# ----------------------------
# Run experiments with LOOCV
# ----------------------------
results = []
exp_id = 0

for mode in preproc_modes:
    for region_name, start_idx, end_idx in windows:
        exp_id += 1
        X_list, y_list, infos = [], [], []
        for pid in pids_sorted:
            spec = data_by_pid[pid]['spectrum'][start_idx:end_idx]
            spec_p = preprocess_vector(spec, mode)
            if spec_p.size == 0:
                continue
            X_list.append(spec_p)
            y_list.append(data_by_pid[pid]['label'])
            infos.append((pid, data_by_pid[pid]['n_files']))
        X_arr = np.vstack(X_list)
        y_arr = np.array(y_list)
        n_samples, n_features = X_arr.shape[0], X_arr.shape[1]

        if n_samples < 2:
            print(f"Experiment {exp_id} ({mode}, {region_name}) skipped: less samples ({n_samples}).")
            continue

        clf = RandomForestClassifier(n_estimators=n_estimators, random_state=random_state)

        # LOOCV
        loo = LeaveOneOut()
        y_true_all, y_pred_all = [], []
        for train_idx, test_idx in loo.split(X_arr):
            X_train, X_test = X_arr[train_idx], X_arr[test_idx]
            y_train, y_test = y_arr[train_idx], y_arr[test_idx]
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            y_true_all.extend(y_test.tolist())
            y_pred_all.extend(y_pred.tolist())

        # metrics
        acc = accuracy_score(y_true_all, y_pred_all)
        prec, rec, f1, _ = precision_recall_fscore_support(y_true_all, y_pred_all, average='macro', zero_division=0)
        cls_report = classification_report(y_true_all, y_pred_all, zero_division=0)
        cm = confusion_matrix(y_true_all, y_pred_all)

        # Raman range
        wavenumber = data_by_pid[pids_sorted[0]]['wavenumber']
        raman_start = wavenumber[start_idx]
        raman_end = wavenumber[end_idx-1]
        raman_range_str = f"{raman_start:.1f}–{raman_end:.1f} cm⁻¹"

        # save report & CM figure
        exp_name = f"exp_{exp_id:03d}_{mode}_{region_name}"
        rep_file = os.path.join(output_dir, f"report_{exp_name}.txt")
        with open(rep_file, "w", encoding="utf-8") as fh:
            fh.write(f"Experiment: {exp_name}\n")
            fh.write(f"Normalization: {mode}\n")
            fh.write(f"Region: {region_name} (indices {start_idx}:{end_idx})\n")
            fh.write(f"RamanRegion: {raman_range_str}\n")
            fh.write(f"n_samples: {n_samples}, n_features: {n_features}\n\n")
            fh.write(f"Accuracy: {acc:.4f}\n")
            fh.write(f"Precision (macro): {prec:.4f}\n")
            fh.write(f"Recall (macro): {rec:.4f}\n")
            fh.write(f"F1 (macro): {f1:.4f}\n\n")
            fh.write("Classification report:\n")
            fh.write(cls_report)
            fh.write("\nConfusion matrix (raw):\n")
            fh.write(np.array2string(cm))

        plt.figure(figsize=(6,5))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                    xticklabels=[f"D <= {limit_D}", f"D > {limit_D}"],
                    yticklabels=[f"D <= {limit_D}", f"D > {limit_D}"])
        plt.xlabel("Predicted label")
        plt.ylabel("True label")
        plt.title(
            f"RandomForest LOOCV\n"
            f"Excitation wavelength: 1064nm\n"
            f"Normalization: {mode}\n"
            f"Raman region: {raman_range_str}\n"
            f"Sample averaging: yes\n"
            f"Accuracy: {acc:.4f}"
        )
        plt.tight_layout()
        cm_file = os.path.join(output_dir, f"cm_{exp_name}.png")
        plt.savefig(cm_file)
        plt.close()

        # collect summary
        results.append({
            'exp_id': exp_id,
            'exp_name': exp_name,
            'Normalization': mode,
            'region': region_name,
            'region_start': int(start_idx),
            'region_end': int(end_idx),
            'raman_start': float(raman_start),
            'raman_end': float(raman_end),
            'n_samples': int(n_samples),
            'n_features': int(n_features),
            'accuracy': float(acc),
            'precision_macro': float(prec),
            'recall_macro': float(rec),
            'f1_macro': float(f1),
            'report_file': rep_file,
            'cm_file': cm_file
        })

        print(f"[{exp_id:03d}] mode={mode} region={region_name} samples={n_samples} features={n_features} acc={acc:.3f}")

# ----------------------------
# Save summary CSV
# ----------------------------
df_results = pd.DataFrame(results)
summary_csv = os.path.join(output_dir, "results_summary_loocv.csv")
df_results.to_csv(summary_csv, index=False)
print("Summary saved to:", summary_csv)
print("Individual reports and confusion matrices in the directory:", output_dir)


80 files found by pattern 'P*_A*_D*_Z*_F*_1064nm_*mW_*ms_*av_no*.csv'.
20 P groups found.


C:\Users\miles\AppData\Local\Temp\ipykernel_20380\2061345168.py:22: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  z = spsolve(Z, w * y)
C:\Users\miles\AppData\Local\Temp\ipykernel_20380\2061345168.py:22: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  z = spsolve(Z, w * y)
C:\Users\miles\AppData\Local\Temp\ipykernel_20380\2061345168.py:22: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  z = spsolve(Z, w * y)
C:\Users\miles\AppData\Local\Temp\ipykernel_20380\2061345168.py:22: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  z = spsolve(Z, w * y)
C:\Users\miles\AppData\Local\Temp\ipykernel_20380\2061345168.py:22: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  z = spsolve(Z, w * y)
C:\Users\miles\AppData\Local\Temp\ipykernel_20380\2061345168.py:22: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  z = spsolve(Z, w * y)
C:\Users\miles\A

20 averaged group spectra are generated.
Total regions: 41 (1 full + 40 windows). Each vector length: 512
[001] mode=none region=full samples=20 features=512 acc=0.650
[002] mode=none region=win_1 samples=20 features=13 acc=0.500
[003] mode=none region=win_2 samples=20 features=13 acc=0.600
[004] mode=none region=win_3 samples=20 features=12 acc=0.500
[005] mode=none region=win_4 samples=20 features=13 acc=0.700
[006] mode=none region=win_5 samples=20 features=13 acc=0.550
[007] mode=none region=win_6 samples=20 features=13 acc=0.450
[008] mode=none region=win_7 samples=20 features=13 acc=0.600
[009] mode=none region=win_8 samples=20 features=12 acc=0.450
[010] mode=none region=win_9 samples=20 features=13 acc=0.650
[011] mode=none region=win_10 samples=20 features=13 acc=0.800
[012] mode=none region=win_11 samples=20 features=13 acc=0.650
[013] mode=none region=win_12 samples=20 features=13 acc=0.600
[014] mode=none region=win_13 samples=20 features=12 acc=0.700
[015] mode=none region

## FUM

-Random forest <br>
-Full spectrum and 40 partial spectra <br>
-4 measurements of the same sample were averaged <br>
-Baseline subtracted <br>
-Normalization: none, max, integral

In [5]:
# ----------------------------
# Filename and Parameters
# ----------------------------
file_pattern = "P*_A*_D*_Z*_F*_1064nm_*mW_*ms_*av_no*.csv"  
limit_F = 1500  # Threshold for Fum
random_state = 42
n_estimators = 200

output_dir = "Results_Random-Forest_sampels-averaged_Fum_1064nm"  #Defines folder for results
os.makedirs(output_dir, exist_ok=True)

# ----------------------------
# Baseline ALS 
# ----------------------------
def baseline_als(y, lam=1e5, p=0.001, niter=10):
    L = len(y)
    D = sparse.diags([1, -2, 1], [0, 1, 2], shape=(L-2, L))
    w = np.ones(L)
    for i in range(niter):
        W = sparse.spdiags(w, 0, L, L)
        Z = W + lam * (D.T @ D)
        z = spsolve(Z, w * y)
        w = p * (y > z) + (1.0 - p) * (y < z)
    return z

# ----------------------------
# Extraction P- and F-Values
# ----------------------------
def extract_P_value_from_filename(fname):
    base = os.path.basename(fname)
    m = re.search(r'(P\d+)', base)
    return m.group(1) if m else None

def extract_F_from_filename(fname):
    base = os.path.basename(fname)
    parts = base.replace(".csv","").split("_")
    for token in parts:
        if token.startswith("F"):
            try:
                return float(token[1:])
            except:
                pass
    return None

# ----------------------------
# Collect files and group by P
# ----------------------------
files = sorted(glob.glob(file_pattern))
print(f"{len(files)} files found by pattern '{file_pattern}'.")

groups = defaultdict(list)
for f in files:
    pid = extract_P_value_from_filename(f)
    if pid is None:
        print("Warning: File without P-token:", f)
        continue
    groups[pid].append(f)

print(f"{len(groups)} P groups found.")

# ----------------------------
# Load spectra, baseline-remove, average per group
# ----------------------------
group_spectra = {}
for pid, filelist in groups.items():
    spectra_list = []
    wav = None
    label = None
    for f in filelist:
        df = pd.read_csv(f, skiprows=56, header=None)
        try:
            df = df[[2,3]].apply(pd.to_numeric, errors='coerce').dropna()
            df.columns = ["Wavenumber", "Processed"]
        except Exception as e:
            raise RuntimeError(f"Error reading {f}: {e}")
        wavenumbers = df["Wavenumber"].values
        proc = df["Processed"].values.astype(float)

        if wav is None:
            wav = wavenumbers
        else:
            if not np.allclose(wav, wavenumbers):
                proc = np.interp(wav, wavenumbers, proc)

        baseline = baseline_als(proc, lam=1e3, p=0.001, niter=10)
        proc_corr = proc - baseline
        spectra_list.append(proc_corr)

        if label is None:
            fval = extract_F_from_filename(f)
            if fval is None:
                raise RuntimeError(f"No F specification in file name {f}.")
            label = int(fval > limit_F)

    spectra_arr = np.vstack(spectra_list)
    mean_spec = spectra_arr.mean(axis=0)
    group_spectra[pid] = {'wavenumber': wav, 'spectrum': mean_spec, 'label': label, 'n_original_files': len(filelist)}

print(f"{len(group_spectra)} averaged group spectra are generated.")

# ----------------------------
# Prepare regions: full + 40 windows
# ----------------------------
any_pid = next(iter(group_spectra))
L = len(group_spectra[any_pid]['spectrum'])
windows = [('full', 0, L)]
n_win = 40
inds = np.round(np.linspace(0, L, n_win+1)).astype(int)
for i in range(n_win):
    windows.append((f'win_{i+1}', inds[i], inds[i+1]))

print(f"Total regions: {len(windows)} (1 full + {n_win} windows). Each vector length: {L}")

# ----------------------------
# Preprocessing modes
# ----------------------------
def preprocess_vector(v, mode):
    if mode == 'none':
        return v.copy()
    elif mode == 'max':
        m = np.max(np.abs(v))
        if m == 0:
            return v.copy()
        return v / m
    elif mode == 'integral':
        s = np.sum(np.abs(v))
        if s == 0:
            return v.copy()
        return v / s
    else:
        raise ValueError("Unknown mode")

preproc_modes = ['none', 'max', 'integral']

# ----------------------------
# Build dataset arrays
# ----------------------------
pids_sorted = sorted(group_spectra.keys())
data_by_pid = {}
for pid in pids_sorted:
    data_by_pid[pid] = {
        'wavenumber': group_spectra[pid]['wavenumber'],
        'spectrum': group_spectra[pid]['spectrum'],
        'label': group_spectra[pid]['label'],
        'n_files': group_spectra[pid]['n_original_files']
    }

# ----------------------------
# Run experiments with LOOCV
# ----------------------------
results = []
exp_id = 0

for mode in preproc_modes:
    for region_name, start_idx, end_idx in windows:
        exp_id += 1
        X_list, y_list, infos = [], [], []
        for pid in pids_sorted:
            spec = data_by_pid[pid]['spectrum'][start_idx:end_idx]
            spec_p = preprocess_vector(spec, mode)
            if spec_p.size == 0:
                continue
            X_list.append(spec_p)
            y_list.append(data_by_pid[pid]['label'])
            infos.append((pid, data_by_pid[pid]['n_files']))
        X_arr = np.vstack(X_list)
        y_arr = np.array(y_list)
        n_samples, n_features = X_arr.shape[0], X_arr.shape[1]

        if n_samples < 2:
            print(f"Experiment {exp_id} ({mode}, {region_name}) skipped: less samples ({n_samples}).")
            continue

        clf = RandomForestClassifier(n_estimators=n_estimators, random_state=random_state)

        # LOOCV
        loo = LeaveOneOut()
        y_true_all, y_pred_all = [], []
        for train_idx, test_idx in loo.split(X_arr):
            X_train, X_test = X_arr[train_idx], X_arr[test_idx]
            y_train, y_test = y_arr[train_idx], y_arr[test_idx]
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            y_true_all.extend(y_test.tolist())
            y_pred_all.extend(y_pred.tolist())

        # metrics
        acc = accuracy_score(y_true_all, y_pred_all)
        prec, rec, f1, _ = precision_recall_fscore_support(y_true_all, y_pred_all, average='macro', zero_division=0)
        cls_report = classification_report(y_true_all, y_pred_all, zero_division=0)
        cm = confusion_matrix(y_true_all, y_pred_all)

        # Raman range
        wavenumber = data_by_pid[pids_sorted[0]]['wavenumber']
        raman_start = wavenumber[start_idx]
        raman_end = wavenumber[end_idx-1]
        raman_range_str = f"{raman_start:.1f}–{raman_end:.1f} cm⁻¹"

        # save report & CM figure
        exp_name = f"exp_{exp_id:03d}_{mode}_{region_name}"
        rep_file = os.path.join(output_dir, f"report_{exp_name}.txt")
        with open(rep_file, "w", encoding="utf-8") as fh:
            fh.write(f"Experiment: {exp_name}\n")
            fh.write(f"Normalization: {mode}\n")
            fh.write(f"Region: {region_name} (indices {start_idx}:{end_idx})\n")
            fh.write(f"RamanRegion: {raman_range_str}\n")
            fh.write(f"n_samples: {n_samples}, n_features: {n_features}\n\n")
            fh.write(f"Accuracy: {acc:.4f}\n")
            fh.write(f"Precision (macro): {prec:.4f}\n")
            fh.write(f"Recall (macro): {rec:.4f}\n")
            fh.write(f"F1 (macro): {f1:.4f}\n\n")
            fh.write("Classification report:\n")
            fh.write(cls_report)
            fh.write("\nConfusion matrix (raw):\n")
            fh.write(np.array2string(cm))

        plt.figure(figsize=(6,5))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                    xticklabels=[f"F <= {limit_F}", f"F > {limit_F}"],
                    yticklabels=[f"F <= {limit_F}", f"F > {limit_F}"])
        plt.xlabel("Predicted label")
        plt.ylabel("True label")
        plt.title(
            f"RandomForest LOOCV\n"
            f"Excitation wavelength: 1064nm\n"
            f"Normalization: {mode}\n"
            f"Raman region: {raman_range_str}\n"
            f"Sample averaging: yes\n"
            f"Accuracy: {acc:.4f}"
        )
        plt.tight_layout()
        cm_file = os.path.join(output_dir, f"cm_{exp_name}.png")
        plt.savefig(cm_file)
        plt.close()

        # collect summary
        results.append({
            'exp_id': exp_id,
            'exp_name': exp_name,
            'Normalization': mode,
            'region': region_name,
            'region_start': int(start_idx),
            'region_end': int(end_idx),
            'raman_start': float(raman_start),
            'raman_end': float(raman_end),
            'n_samples': int(n_samples),
            'n_features': int(n_features),
            'accuracy': float(acc),
            'precision_macro': float(prec),
            'recall_macro': float(rec),
            'f1_macro': float(f1),
            'report_file': rep_file,
            'cm_file': cm_file
        })

        print(f"[{exp_id:03d}] mode={mode} region={region_name} samples={n_samples} features={n_features} acc={acc:.3f}")

# ----------------------------
# Save summary CSV
# ----------------------------
df_results = pd.DataFrame(results)
summary_csv = os.path.join(output_dir, "results_summary_loocv.csv")
df_results.to_csv(summary_csv, index=False)
print("Summary saved to:", summary_csv)
print("Individual reports and confusion matrices in the directory:", output_dir)


80 files found by pattern 'P*_A*_D*_Z*_F*_1064nm_*mW_*ms_*av_no*.csv'.
20 P groups found.


C:\Users\miles\AppData\Local\Temp\ipykernel_20380\1616899868.py:22: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  z = spsolve(Z, w * y)
C:\Users\miles\AppData\Local\Temp\ipykernel_20380\1616899868.py:22: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  z = spsolve(Z, w * y)
C:\Users\miles\AppData\Local\Temp\ipykernel_20380\1616899868.py:22: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  z = spsolve(Z, w * y)
C:\Users\miles\AppData\Local\Temp\ipykernel_20380\1616899868.py:22: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  z = spsolve(Z, w * y)
C:\Users\miles\AppData\Local\Temp\ipykernel_20380\1616899868.py:22: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  z = spsolve(Z, w * y)
C:\Users\miles\AppData\Local\Temp\ipykernel_20380\1616899868.py:22: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  z = spsolve(Z, w * y)
C:\Users\miles\A

20 averaged group spectra are generated.
Total regions: 41 (1 full + 40 windows). Each vector length: 512
[001] mode=none region=full samples=20 features=512 acc=0.900
[002] mode=none region=win_1 samples=20 features=13 acc=0.900
[003] mode=none region=win_2 samples=20 features=13 acc=0.850
[004] mode=none region=win_3 samples=20 features=12 acc=0.850
[005] mode=none region=win_4 samples=20 features=13 acc=0.850
[006] mode=none region=win_5 samples=20 features=13 acc=0.850
[007] mode=none region=win_6 samples=20 features=13 acc=0.900
[008] mode=none region=win_7 samples=20 features=13 acc=0.900
[009] mode=none region=win_8 samples=20 features=12 acc=0.800
[010] mode=none region=win_9 samples=20 features=13 acc=0.850
[011] mode=none region=win_10 samples=20 features=13 acc=0.900
[012] mode=none region=win_11 samples=20 features=13 acc=0.900
[013] mode=none region=win_12 samples=20 features=13 acc=0.800
[014] mode=none region=win_13 samples=20 features=12 acc=0.850
[015] mode=none region